# 检索机器人

## Step1 读取faq数据

In [2]:
import pandas as pd

data = pd.read_csv("./law_faq.csv")
data.head()

,title,reply
0,在法律中定金与订金的区别订金和定金哪个受,“定金”是指当事人约定由一方向对方给付的，作为债权担保的一定数额的货币，它属于一种法律上的担...
1,盗窃罪的犯罪客体是什么，盗窃罪的犯罪主体,盗窃罪的客体要件本罪侵犯的客体是公私财物的所有权。侵犯的对象，是国家、集体或个人的财物，一般...
2,非法微整形机构构成非法经营罪吗,符合要件就有可能。非法经营罪，是指未经许可经营专营、专卖物品或其他限制买卖的物品，买卖进出口...
3,入室持刀行凶伤人能不能判刑,对于入室持刀伤人涉嫌故意伤害刑事犯罪，一经定罪，故意伤害他人身体的，处三年以下有期徒刑、拘役...
4,对交通事故责任认定书不服怎么办，交通事故损,事故认定书下发后，如果你对认定不满意，可在接到认定书3日内到上一级公安机关复议。


## Step2 加载模型

In [ ]:
from dual_model import DualModel

# 需要完成前置模型训练
dual_model = DualModel.from_pretrained("../12-sentence_similarity/dual_model/checkpoint-500/")
dual_model = dual_model.cuda()
dual_model.eval()
print("匹配模型加载成功！")

In [ ]:
from transformers import AutoTokenizer
tokenzier = AutoTokenizer.from_pretrained("hfl/chinese-macbert-base")
tokenzier

## Step3 将问题编码为向量

In [ ]:
import torch
from tqdm import tqdm
questions = data["title"].to_list()
vectors = []
with torch.inference_mode():
    #遍历tqdm的每一个索引，每32个（步长）为一次循环
    for i in tqdm(range(0, len(questions), 32)):
        #为了快速处理防止内存爆炸，设置批次，每32个为一个batch（batch_size)
        batch_sens = questions[i: i + 32]
        inputs = tokenzier(batch_sens, return_tensors="pt", padding=True, max_length=128, truncation=True)
        inputs = {k: v.to(dual_model.device) for k, v in inputs.items()}
        '''
        k：指的是 inputs 字典中的键（key）
        v：指的是 inputs 字典中的值（value）
        inputs = {
            'input_ids': tensor([[101, 123, 456, 102, 0, 0]]),
            'attention_mask': tensor([[1, 1, 1, 1, 0, 0]]),
            'token_type_ids': tensor([[0, 0, 0, 0, 0, 0]])}
        第一次迭代：k = 'input_ids', v = tensor([[101, 123, 456, 102, 0, 0]])
        第二次迭代：k = 'attention_mask', v = tensor([[1, 1, 1, 1, 0, 0]])
        第三次迭代：k = 'token_type_ids', v = tensor([[0, 0, 0, 0, 0, 0]])
        '''
        vector = dual_model.bert(**inputs)[1]
        '''
        ①dual_model.bert()是DualModel 中定义的 BERT 模型实例即
         self.bert = BertModel(config)
        ②**inputs：将字典解包为关键字参数，相当于：dual_model.bert(input_ids=..., attention_mask=..., token_type_ids=...)
        ③[1] - 索引操作：由于 bert 模型返回的是一个元组，通常包含多个输出，1表示第二个元素，即池化输出

        '''
        vectors.append(vector)
#沿第0维度（批次维度）将vectors里面包含的多个张量进行拼接并转换成numpy数组
vectors = torch.concat(vectors, dim=0).cpu().numpy()
vectors.shape

## Step4 创建索引

In [ ]:
import faiss

index = faiss.IndexFlatIP(768)
'''#创建一个使用 内积（Inner Product） 作为相似度度量的平面索引。
768：向量的维度（你的BERT输出维度）
IndexFlatIP：精确的暴力搜索索引，使用内积计算相似度'''
faiss.normalize_L2(vectors) #对向量进行 L2归一化，使每个向量的L2范数(向量模长)等于1。
index.add(vectors)
index

## Step5 对问题进行向量编码

In [ ]:
quesiton = "寻衅滋事"
with torch.inference_mode():
    inputs = tokenzier(quesiton, return_tensors="pt", padding=True, max_length=128, truncation=True)
    inputs = {k: v.to(dual_model.device) for k, v in inputs.items()}
    vector = dual_model.bert(**inputs)[1]
    q_vector = vector.cpu().numpy()
q_vector.shape

## Step6 （用step5生成的查询向量进行相似度搜索）向量匹配(召回)

In [ ]:
faiss.normalize_L2(q_vector)
#从index里查询向量q_vector返回最相似的10个结果，得到相似度分数scores和对应的indexes
scores, indexes = index.search(q_vector, 10)
topk_result = data.values[indexes[0].tolist()]
topk_result[:, 0]

## Step7 加载交互模型

In [ ]:
from transformers import BertForSequenceClassification

# 需要完成前置模型训练
corss_model = BertForSequenceClassification.from_pretrained("../12-sentence_similarity/cross_model/checkpoint-500/")
corss_model = corss_model.cuda()
corss_model.eval()
print("模型加载成功！")

## Step8 最终预测(排序)

In [ ]:
canidate = topk_result[:, 0].tolist()
ques = [quesiton] * len(canidate)
inputs = tokenzier(ques, canidate, return_tensors="pt", padding=True, max_length=128, truncation=True)
inputs = {k: v.to(corss_model.device) for k, v in inputs.items()}
with torch.inference_mode():
    logits = corss_model(**inputs).logits.squeeze()
    result = torch.argmax(logits, dim=-1)
result

In [ ]:
canidate_answer = topk_result[:, 1].tolist()
match_quesiton = canidate[result.item()]
final_answer = canidate_answer[result.item()]
match_quesiton, final_answer